# Forest Fires Regression with Neural Networks

## Setup
### Import all necessary libraries and define constants

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from dataset import MyDataClass
from models import Layer_Dense, Activation_ReLU, Activation_Linear, Loss_MeanSquaredError, Accuracy_Regression, Optimizer_Adam, Model
from preprocess import preprocess_forest_fires


## Data Preprocessing
### Load and preprocess the forest fires data. This section should end with the dataset ready for training.

In [8]:
preprocess_forest_fires()

In [9]:
data = MyDataClass()

# Load and preprocess the forest fires data

In [10]:
data.load_and_preprocess_forest_fires()

## Model Building
### Define and compile the neural network model for regression.

In [11]:
model = Model()

model.add(Layer_Dense(12, 6))  
model.add(Activation_ReLU())
model.add(Layer_Dense(6, 6)) 
model.add(Activation_ReLU())
model.add(Layer_Dense(6, 1)) 
model.add(Activation_Linear())

model.set(
    loss=Loss_MeanSquaredError(),
    optimizer=Optimizer_Adam(learning_rate=1e-3, decay=1e-5),
    accuracy=Accuracy_Regression()
)

model.finalize()

## Training
### Train the model with the forest fires dataset.

In [13]:
print(data.X_train)
print(data.y_train)
print(data.X_test)
print(data.y_test)

[[-2.88472000e-01 -1.09485913e+00  6.69843702e-01 ... -1.05941934e+00
   1.63124094e-03 -6.78170911e-02]
 [-2.88472000e-01 -2.66696454e-01  6.69843702e-01 ... -3.35214892e-01
  -1.00892252e+00 -6.78170911e-02]
 [-1.16128472e+00  5.61466218e-01  2.30025818e-01 ...  1.47588074e-01
   1.63124094e-03 -6.78170911e-02]
 ...
 [-1.16128472e+00 -1.92302180e+00  2.30025818e-01 ...  6.90741409e-01
  -5.03645641e-01 -6.78170911e-02]
 [-1.16128472e+00  5.61466218e-01 -2.09792066e-01 ...  3.88989556e-01
  -1.23349003e+00 -6.78170911e-02]
 [-1.16128472e+00 -2.66696454e-01  2.30025818e-01 ... -2.74864521e-01
   1.63124094e-03 -6.78170911e-02]]
[[3.05269334e-03]
 [2.81434491e-03]
 [2.83267940e-03]
 [1.34575190e-02]
 [5.84870375e-03]
 [0.00000000e+00]
 [1.86095119e-03]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.02856514e-02]
 [0.00000000e+00]
 [6.05955044e-03]
 [1.15507315e-03]
 [0.00000000e+00]
 [5.89453999e-03]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.22122401e-02]
 [4.88614279e-03]
 [3.

In [12]:
model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=25, batch_size=4, print_every=1)


epoch: 1


ZeroDivisionError: division by zero

## Learning Curves
### Plot the learning curves to evaluate the training process.


In [ ]:
def plot_learning_curves(model):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(model.train_acc_history, label='Training Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(model.train_loss_history, label='Training Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_learning_curves(model)

In [ ]:
example_indices = np.random.choice(range(len(data.X_test)), 10)
examples = data.X_test[example_indices]

activations = model.get_activations(examples)


In [ ]:
import seaborn as sns

for i, activation in enumerate(activations):
    plt.figure(figsize=(10, 1))
    sns.heatmap(activation, cmap="viridis", yticklabels=False)
    plt.title(f"Layer {i+1} Activation")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

weights = model.get_weights()
biases = model.get_biases()

num_layers = len(weights)

fig, axes = plt.subplots(num_layers, 2, figsize=(12, num_layers * 4))

for i in range(num_layers):
    ax = axes[i, 0]
    sns.heatmap(weights[i], ax=ax, cmap="viridis")
    ax.set_title(f'Layer {i+1} Weights')

    ax = axes[i, 1]
    sns.heatmap(biases[i].reshape(1, -1), ax=ax, cmap="viridis")
    ax.set_title(f'Layer {i+1} Biases')

plt.tight_layout()
plt.show()

## Hyperparameter search using grid search

In [ ]:
learning_rates = [1e-2, 1e-3, 1e-4]
batch_sizes = [4, 8, 16]
epochs_options = [10, 20, 30]

best_hyperparams = None
best_loss = float('inf')

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for epochs in epochs_options:
            
            model = Model()

            model.add(Layer_Dense(12, 6))  
            model.add(Activation_ReLU())
            model.add(Layer_Dense(6, 6)) 
            model.add(Activation_ReLU())
            model.add(Layer_Dense(6, 1)) 
            model.add(Activation_Linear())
            
            model.set(
                loss=Loss_MeanSquaredError(),
                optimizer=Optimizer_Adam(learning_rate=1e-3, decay=1e-5),
                accuracy=Accuracy_Regression()
            )
            
            model.finalize()

            model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=epochs, batch_size=batch_size)

            validation_loss, validation_accuracy = model.evaluate(data.X_test, data.y_test)

            if validation_loss < best_loss:
                best_loss = validation_loss
                best_hyperparams = {'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs': epochs}

print("Best Hyperparameters:")
print(best_hyperparams)

## Measuring impact of neural network dimensionality

In [ ]:
learning_rate = 0.001
batch_size = 4
epochs = 30

layer_options = [2, 3, 4]
layer_results = []

for num_layers in layer_options:
    model = Model()

    model.add(Layer_Dense(12, 6))  
    model.add(Activation_ReLU())

    for _ in range(num_layers - 1):
        model.add(Layer_Dense(6, 6)) 
        model.add(Activation_ReLU())

    model.add(Layer_Dense(6, 1)) 
    model.add(Activation_Linear())
            
    model.set(
        loss=Loss_MeanSquaredError(),
        optimizer=Optimizer_Adam(learning_rate=1e-3, decay=1e-5),
        accuracy=Accuracy_Regression()
    )
    model.finalize()

    model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=epochs, batch_size=batch_size)

    validation_loss, validation_accuracy = model.evaluate(data.X_test, data.y_test)

    layer_results.append({
        'num_layers': num_layers,
        'loss': validation_loss,
        'accuracy': validation_accuracy
    })

In [ ]:
num_layers_list = [result['num_layers'] for result in layer_results]
losses = [result['loss'] for result in layer_results]
accuracies = [result['accuracy'] for result in layer_results]

# Plotting Loss vs. Number of Layers
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(num_layers_list, losses, marker='o')
plt.title('Validation Loss vs. Number of Layers')
plt.xlabel('Number of Layers')
plt.ylabel('Loss')
plt.grid(True)

# Plotting Accuracy vs. Number of Layers
plt.subplot(1, 2, 2)
plt.plot(num_layers_list, accuracies, marker='o')
plt.title('Validation Accuracy vs. Number of Layers')
plt.xlabel('Number of Layers')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()